# 1. request api
# part 1 of automation test
# send request

In [ ]:
import requests as req

req.get(url, params=None, **kwargs)
# url: URL for the new :class:`Request` object.
# params: (optional) Dictionary, list of tuples or bytes to send in the query string for the :class:`Request`.
# \*\*kwargs: Optional arguments that ``request`` takes.
req.post(url, data=None, json=None, **kwargs)
# url – URL for the new :class:`Request` object.
# data – (optional) Dictionary, list of tuples, bytes, or file-like object to send in the body of the :class:`Request`.
# json – (optional) A JSON serializable Python object to send in the body of the :class:`Request`.
req.put(url, data=None, **kwargs)
# url – URL for the new :class:`Request` object.
# data – (optional) Dictionary, list of tuples, bytes, or file-like object to send in the body of the :class:`Request`.
req.delete(url, **kwargs)
# url – URL for the new :class:`Request` object.

# The above method internally calls the `request` method

req.request(method, url, **kwargs)
# method – method for the new :class:`Request` object: GET, OPTIONS, HEAD, POST, PUT, PATCH, or DELETE.
# url – URL for the new :class:`Request` object.

# `request` method calls the method below (Automatically associating interfaces with cookie associations)
session.request(method=method, url=url, **kwargs)
'''
method,  call for method方式
url,  call for url路径
params=None,  get() call for parameters
data=None,  post(),put() call for parameters
json = None, post() call for parameters
headers=None,  call for headers
cookies=None,  cookies  # cookie: 存储用户会话信息，比如登陆状态，用户标识
files=None,  upload files
-----------------------------------
auth=None,  authentication #authentication: 鉴权，验证用户身份的过程， 确保请求者是其声称的身份
timeout=None,  timeout handling
allow_redirects=True,  whether redirects or not (是否重定向)
proxies=None,  whether set proxies or not (是否设置代理)
hooks=None,  hooks function
stream=None, file download
verify=None,  certification vertify (证书验证)
cert=None,  CA certification
'''

'''
Ex.1 req.get()
step 1: req.get(url, params=None, **kwargs)
step 2: req.request(method=GET, url_r=url, **kwargs=(params=None))
step 3: session.request(method=GET, url_s=url_r, **kwargs=(params=None))
'''

'''
Ex.2 req.post()
step 1: req.post(url, data=None, json=None, **kwargs)
step 2: req.request(method=POST, url_r=url, **kwargs=(data=None, json=None))
step 3: session.request(method=POST, url_s=url_r, **kwargs=(data=None, json=None))
'''

# generate a session object
req.session()

# 2. response api
# part 2 of automation test
# receive response

In [ ]:
import requests as req
from certifi import contents

res = req.get(url, params=None, **kwargs)
print(res.text)  return text information
print(res.json())  return .json
print(res.content)  return byte contents
print(res.status_code)  return status code
print(res.reason) return reason information
print(res.cookies)  return cookies
print(res.encoding)  return encoding e.g: encoding('utf-8')
print(res.headers)  return request headers
print(res.request.(requests data))  return request data

# 3. Test Sample

In [10]:
import random
import requests as req
import jsonpath as jp
import json
import re
class TestApi:
    access_token = ""
    csrf_token = ""
    sess = req.session()
    # 1. get the authentication code access token interface
    def test_get_token(self):
        # ?grant_type=client_credential&appid=appid&secret=secret
        # grant type 用户类型
        # appid 第三方用户唯一id
        # secret 第三方用户唯一密钥
        url = "https://api.weixin.qq.com/cgi-bin/token"
        datas = {
            "grant_type": "client_credential",
            "appid": "wx74a8627810cfa308",
            "secret": "e40a02f9d79a8097df497e6aaf93ab80"
        }

        res = req.get(url, params=datas)
        # return .json -> .json: data consist of {} and []
        # print(res.text)
        result = res.json()
        print(result)

        # get access token
        value = jp.jsonpath(result, "$.access_token")
        # print(value)
        TestApi.access_token = value[0]

    # 2. get the interface of the tag 获取已创建的标签接口
    # Interface association (接口关联): refers to the process of linking or connecting multiple APIs
    '''
    rule of jsonpath expression:
    $ root node
    . child node (sub-node) -> $.access_token
    .. recursion achieve child node
    [] represent get the value of list, start at 0 -> [{id:0}, {id:1}, {id:2}]
    '''
    def test_select_flag(self):
        url = "https://api.weixin.qq.com/cgi-bin/tags/get"
        datas = {
            "access_token": TestApi.access_token
        }
        res = req.get(url, params=datas)
        print(res.json())

    # 3. create the interface of the tag
    def test_create_flag(self):
        url = "https://api.weixin.qq.com/cgi-bin/tags/create?access_token="+TestApi.access_token
        datas = {"tag":{"name":"广东"+str(random.randint(10000,99999))}}
        res = req.post(url, json=datas)
        # print(res.json())
        rs = json.loads(json.dumps(res.json()).replace("\\\\","\\"))
        print(rs)
    
    # 4. delete file
    def test_file_upload(self):
        url = "https://api.weixin.qq.com/cgi-bin/media/upload?acpi.access_token"
        data = {"media":open("/Users/ethan/Downloads/IMG_0184.jpg","rb")}
        res = req.post(url, files=data)
        print(res.json())
        
    # cookie association interface
    def test_start(self):
        url = "http://47.107.116.139/phpwind/"
        res = TestApi.sess.request(method="get",url=url)
        result = res.text  # string
        # print(result)
        TestApi.csrf_token = re.search('name="csrf_token" value="(.*?)"',result).group(1)
        print(TestApi.csrf_token)
        
    def test_login(self):
        url = "http://47.107.116.139/phpwind/index.php?m=u&c=login&a=dorun"
        data = {
            "username":"zhouy218",
            "password":"136671",
            "csrf_token":TestApi.csrf_token,
            "back_url":"http://47.107.116.139/phpwind/",
            "invite":""
        }
        headers = {
            "Accept":"application/json, text/javascript, /; q=0.01",
            "X-Requested-With":"XMLHttpRequest"
        }
        res = TestApi.sess.request(method="post", url=url, data=data, headers=headers)
        print(res.json())

if __name__ == '__main__':
    test = TestApi()
    # test.test_get_token()
    # test.test_select_flag()
    # test.test_create_flag()
    # test.test_file_upload()
    test.test_start()
    test.test_login()

587132f66304b0ee
{'referer': 'http%3A%2F%2F47.107.116.139%2Fphpwind%2Findex.php%3Fm%3Du%26c%3Dlogin%26a%3Dwelcome%26_statu%3Dd2xrMG9xSk52cCUyQjVzYjlsV1VlRW1hY2JVanZjMzlFbWtUR25CUVhHYXJhanhqYnNYeFpxQmtOa3R0NGxObXQ2dFVERFlQcG45MEUlM0R8fA', 'refresh': False, 'state': 'success', 'data': '', 'html': '', 'message': [''], '__error': ''}
